In [36]:
import sys
sys.path.append("/Users/karthika")  # Parent folder

In [37]:
from housing_app_fall25.housing_pipeline import SafeDivide, build_preprocessing

In [38]:
# Install required packages (if not already installed)
!pip install scikit-learn lightgbm xgboost optuna pandas numpy joblib seaborn matplotlib


In [39]:
import pandas as pd
import numpy as np
import joblib
from pathlib import Path

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import f1_score

from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import optuna
import warnings
warnings.filterwarnings("ignore")

In [40]:
import pandas as pd

df = pd.read_csv("../data/housing.csv")
print(df.columns)


Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
      

Step 1: Load Dataset & Create Classification Target

In [41]:
# Load dataset
df = pd.read_csv("../data/housing.csv")  # Adjust path if needed

# Convert to classification: HighPrice if SalePrice > median
median_price = df['SalePrice'].median()
df['HighPrice'] = (df['SalePrice'] > median_price).astype(int)

# Features / target
X = df.drop(columns=['SalePrice', 'HighPrice'])
y = df['HighPrice']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
print(y_train.value_counts())


Train shape: (2344, 81), Test shape: (586, 81)
HighPrice
0    1174
1    1170
Name: count, dtype: int64


In [42]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer


In [43]:
import pandas as pd

# Check how many unique values each feature has
print(df.nunique())

# Check for missing values
print(df.isnull().sum())

# Basic descriptive statistics
print(df.describe())


Order             2930
PID               2930
MS SubClass         16
MS Zoning            7
Lot Frontage       128
                  ... 
Yr Sold              5
Sale Type           10
Sale Condition       6
SalePrice         1032
HighPrice            2
Length: 83, dtype: int64
Order               0
PID                 0
MS SubClass         0
MS Zoning           0
Lot Frontage      490
                 ... 
Yr Sold             0
Sale Type           0
Sale Condition      0
SalePrice           0
HighPrice           0
Length: 83, dtype: int64
            Order           PID  MS SubClass  Lot Frontage       Lot Area  \
count  2930.00000  2.930000e+03  2930.000000   2440.000000    2930.000000   
mean   1465.50000  7.144645e+08    57.387372     69.224590   10147.921843   
std     845.96247  1.887308e+08    42.638025     23.365335    7880.017759   
min       1.00000  5.263011e+08    20.000000     21.000000    1300.000000   
25%     733.25000  5.284770e+08    20.000000     58.000000    7440.250

Step 2: Define Preprocessing Pipeline

In [44]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Identify numeric and categorical columns
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

# Numeric pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical pipeline
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Full preprocessor (NO SafeDivide)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)


# SafeDivide transformer
from sklearn.base import BaseEstimator, TransformerMixin

class SafeDivide(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        numerator = X_copy.iloc[:, 0].fillna(0)
        denominator = X_copy.iloc[:, 1].replace(0,1).fillna(1)
        return (numerator / denominator).values.reshape(-1,1)

# Identify numeric and categorical columns
numeric_features = X_train.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

# Numeric pipeline
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical pipeline
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Full preprocessor with SafeDivide (example using first two numeric cols)
safe_divider = Pipeline([('safe_divide', SafeDivide())])
preprocessor = ColumnTransformer([
    ('safe_div', safe_divider, numeric_features[:2]),
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])


Step 3: Define Classifiers & Experiment Configurations

In [45]:
# 4 classifiers
classifiers = {
    "Ridge": RidgeClassifier(),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42)
}

# 4 conditions
conditions = [
    {"PCA": False, "Optuna": False},
    {"PCA": False, "Optuna": True},
    {"PCA": True, "Optuna": False},
    {"PCA": True, "Optuna": True}
]

# Folder to save models
model_dir = Path("../api/models")
model_dir.mkdir(parents=True, exist_ok=True)

# Log for results
results = []


Step 4: Define Optuna Objective

In [46]:
def objective(trial, classifier_name, X_train, y_train, use_pca):
    # Hyperparameters
    if classifier_name == "GradientBoosting":
        n_estimators = trial.suggest_int("n_estimators", 50, 300)
        learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
        clf = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=42)
    elif classifier_name == "XGBoost":
        n_estimators = trial.suggest_int("n_estimators", 50, 300)
        max_depth = trial.suggest_int("max_depth", 3, 10)
        learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
        clf = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate,
                            use_label_encoder=False, eval_metric='logloss', random_state=42)
    elif classifier_name == "LightGBM":
        n_estimators = trial.suggest_int("n_estimators", 50, 300)
        learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
        clf = LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=42)
    else:  # Ridge
        alpha = trial.suggest_float("alpha", 0.1, 10.0)
        clf = RidgeClassifier(alpha=alpha)
    
    steps = [('preprocessor', preprocessor)]
    if use_pca:
        steps.append(('svd', TruncatedSVD(n_components=min(100, X_train.shape[1]))))
    steps.append(('classifier', clf))
    
    pipeline = Pipeline(steps)
    
    score = cross_val_score(pipeline, X_train, y_train, cv=3, scoring='f1').mean()
    return score


Step 5: Run 16 Experiments

In [47]:
import warnings
warnings.filterwarnings("ignore")

for clf_name, clf in classifiers.items():
    for cond in conditions:
        use_pca = cond["PCA"]
        use_optuna = cond["Optuna"]
        experiment_name = f"{clf_name}_{'PCA' if use_pca else 'NoPCA'}_{'Optuna' if use_optuna else 'NoOptuna'}"
        print(f"Running experiment: {experiment_name}")
        
        if use_optuna:
            study = optuna.create_study(direction="maximize")
            study.optimize(lambda trial: objective(trial, clf_name, X_train, y_train, use_pca), n_trials=20)
            best_params = study.best_params
            print("Best params:", best_params)
            
            # Build classifier with best params
            if clf_name == "GradientBoosting":
                clf = GradientBoostingClassifier(**best_params, random_state=42)
            elif clf_name == "XGBoost":
                clf = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss', random_state=42)
            elif clf_name == "LightGBM":
                clf = LGBMClassifier(random_state=42, **best_params)
            else:
                clf = RidgeClassifier(**best_params)
        
        # Build pipeline
        steps = [('preprocessor', preprocessor)]
        if use_pca:
            steps.append(('svd', TruncatedSVD(n_components=min(100, X_train.shape[1]))))
        steps.append(('classifier', clf))
        pipeline = Pipeline(steps)
        
        # Train full training set
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        test_f1 = f1_score(y_test, y_pred)
        
        # Save model
        model_file = model_dir / f"{experiment_name}.pkl"
        joblib.dump(pipeline, model_file)
        
        # Log results
        results.append({
            "Experiment": experiment_name,
            "Classifier": clf_name,
            "PCA": use_pca,
            "Optuna": use_optuna,
            "Test F1-score": test_f1,
            "Model Path": str(model_file)
        })
        print(f"{experiment_name} -> F1: {test_f1:.4f}, saved to {model_file}")


[I 2025-12-18 10:04:48,032] A new study created in memory with name: no-name-5f3e55dc-cc15-4b63-86f6-dfa9a0da3722
[I 2025-12-18 10:04:48,134] Trial 0 finished with value: 0.9172062948699663 and parameters: {'alpha': 4.520266490964633}. Best is trial 0 with value: 0.9172062948699663.


Running experiment: Ridge_NoPCA_NoOptuna
Ridge_NoPCA_NoOptuna -> F1: 0.9210, saved to ../api/models/Ridge_NoPCA_NoOptuna.pkl
Running experiment: Ridge_NoPCA_Optuna


[I 2025-12-18 10:04:48,246] Trial 1 finished with value: 0.9162528189036343 and parameters: {'alpha': 2.6030475429524333}. Best is trial 0 with value: 0.9172062948699663.
[I 2025-12-18 10:04:48,345] Trial 2 finished with value: 0.9163435605895162 and parameters: {'alpha': 6.318025563044053}. Best is trial 0 with value: 0.9172062948699663.
[I 2025-12-18 10:04:48,446] Trial 3 finished with value: 0.9171993586211807 and parameters: {'alpha': 4.883396228577902}. Best is trial 0 with value: 0.9172062948699663.
[I 2025-12-18 10:04:48,543] Trial 4 finished with value: 0.9158536988772038 and parameters: {'alpha': 9.66332565630756}. Best is trial 0 with value: 0.9172062948699663.
[I 2025-12-18 10:04:48,644] Trial 5 finished with value: 0.9163435605895162 and parameters: {'alpha': 7.201420428593681}. Best is trial 0 with value: 0.9172062948699663.
[I 2025-12-18 10:04:48,751] Trial 6 finished with value: 0.9166495435711043 and parameters: {'alpha': 3.0765960139142416}. Best is trial 0 with value:

Best params: {'alpha': 4.520266490964633}
Ridge_NoPCA_Optuna -> F1: 0.9262, saved to ../api/models/Ridge_NoPCA_Optuna.pkl
Running experiment: Ridge_PCA_NoOptuna
Ridge_PCA_NoOptuna -> F1: 0.9110, saved to ../api/models/Ridge_PCA_NoOptuna.pkl
Running experiment: Ridge_PCA_Optuna


[I 2025-12-18 10:04:50,499] Trial 0 finished with value: 0.9057185883922911 and parameters: {'alpha': 2.1660692656043863}. Best is trial 0 with value: 0.9057185883922911.
[I 2025-12-18 10:04:50,670] Trial 1 finished with value: 0.9052329391827411 and parameters: {'alpha': 4.581993484040073}. Best is trial 0 with value: 0.9057185883922911.
[I 2025-12-18 10:04:50,842] Trial 2 finished with value: 0.9056268934422288 and parameters: {'alpha': 2.005609893729005}. Best is trial 0 with value: 0.9057185883922911.
[I 2025-12-18 10:04:51,015] Trial 3 finished with value: 0.9052329391827411 and parameters: {'alpha': 7.6250461258528865}. Best is trial 0 with value: 0.9057185883922911.
[I 2025-12-18 10:04:51,186] Trial 4 finished with value: 0.9062020276604996 and parameters: {'alpha': 4.292212485919793}. Best is trial 4 with value: 0.9062020276604996.
[I 2025-12-18 10:04:51,357] Trial 5 finished with value: 0.9062031894933608 and parameters: {'alpha': 7.932418620010195}. Best is trial 5 with value

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-12-18 10:10:59,401] Trial 18 finished with value: 0.9306117716842851 and parameters: {'n_estimators': 110, 'learning_rate': 0.2929779406417012}. Best is trial 17 with value: 0.931198327547014.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-12-18 10:11:00,024] Trial 19 finished with value: 0.9323577113465574 and parameters: {'n_estimators': 109, 'learning_rate': 0.26149966770541166}. Best is trial 19 with value: 0.9323577113465574.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

Step 6: Save Results to CSV

In [54]:
results_df = pd.DataFrame(results)
results_df.sort_values("Test F1-score", inplace=True)

results_df.to_csv(model_dir / "experiment_results.csv", index=False)
results_df.head(16)


,Experiment,Classifier,PCA,Optuna,Test F1-score,Model Path
2,Ridge_PCA_NoOptuna,Ridge,True,False,0.910959,../api/models/Ridge_PCA_NoOptuna.pkl
3,Ridge_PCA_Optuna,Ridge,True,True,0.910959,../api/models/Ridge_PCA_Optuna.pkl
6,GradientBoosting_PCA_NoOptuna,GradientBoosting,True,False,0.914676,../api/models/GradientBoosting_PCA_NoOptuna.pkl
8,XGBoost_NoPCA_NoOptuna,XGBoost,False,False,0.919932,../api/models/XGBoost_NoPCA_NoOptuna.pkl
0,Ridge_NoPCA_NoOptuna,Ridge,False,False,0.920962,../api/models/Ridge_NoPCA_NoOptuna.pkl
7,GradientBoosting_PCA_Optuna,GradientBoosting,True,True,0.921769,../api/models/GradientBoosting_PCA_Optuna.pkl
9,XGBoost_NoPCA_Optuna,XGBoost,False,True,0.924658,../api/models/XGBoost_NoPCA_Optuna.pkl
13,LightGBM_NoPCA_Optuna,LightGBM,False,True,0.925424,../api/models/LightGBM_NoPCA_Optuna.pkl
1,Ridge_NoPCA_Optuna,Ridge,False,True,0.926244,../api/models/Ridge_NoPCA_Optuna.pkl
11,XGBoost_PCA_Optuna,XGBoost,True,True,0.926746,../api/models/XGBoost_PCA_Optuna.pkl
